# <font size=35 color=lightgreen>** Sentiment API **<font>
---

### <font size=12 color=lightgreen>Configuración Inicial (Librerías)</font>

#### 1. Procesamiento y Manipulación de Datos
* **`pandas`**
    * Nos ayuda con la manipulación y análisis de datos estructurados.
    * Carga el dataset (CSV), gestiona el DataFrame y permite filtrar o limpiar registros.
* **`numpy`**
    * Realiza las operaciones matemáticas y manejo de arrays eficientes.
    * Soporte numérico fundamental para las transformaciones vectoriales de los textos.

#### 2. Visualización y Análisis Exploratorio

* **`matplotlib.pyplot`**
    * Generación de gráficos estáticos.
    * Visualización básica de la distribución de clases (Positivo vs. Negativo).
* **`seaborn`**
    * Visualización de datos estadísticos avanzada.
    * Generación de matrices de confusión y gráficos de distribución estéticos para la presentación.

#### 3. Procesamiento de Lenguaje Natural (NLP) y Limpieza

* **`re`** (Regular Expressions)
    * Manejo de expresiones regulares.
    * Eliminación de ruido en el texto: URLs, menciones (@usuario), hashtags (#) y caracteres especiales no alfanuméricos.
* **`string`**
    * Constantes de cadenas comunes.
    * Provee listas estándar de signos de puntuación para su eliminación eficiente.

#### 4. Modelado y Machine Learning (Core)

* **`scikit-learn`**
    * Biblioteca principal de Machine Learning.
    * **`TfidfVectorizer`**: Transforma el texto limpio en vectores numéricos.
    * **`LogisticRegression`**: Algoritmo de clasificación supervisada.
    * **`metrics`**: Cálculo de precisión, recall y F1-score.
    * **`Pipeline`**: Encapsulamiento de los pasos de transformación y predicción.

#### 5. Persistencia e Integración
Herramientas para conectar el modelo con el Backend.

* **`joblib`**
    * Serialización eficiente de objetos Python.
    * Exportar (`dump`) el pipeline entrenado a un archivo `.joblib` y cargarlo (`load`) en la API para realizar predicciones.
* **`fastapi` & `uvicorn`**
    * Framework web moderno de alto rendimiento.
    * Exponer el modelo entrenado como un microservicio REST (endpoint `/predict`) para ser consumido por el Backend en Java.




---



### <font size=16  color=lightgreen> Importando librerías <font>



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import uvicorn
import sklearn
import fastapi
import joblib
import warnings
import unicodedata
import nltk
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

### <font size = 8 color="lightgreen">Importación del dataset <font>

In [ ]:
# URL del archivo RAW en GitHub 
url_dataset = 'https://github.com/ml-punto-tech/sentiment-api/raw/refs/heads/main/data-science/datasets/sentimentdataset_es.csv'

# Cargar directamente usando pandas
# Mantenemos tus parámetros de encoding y separador
try:
    df = pd.read_csv(url_dataset, encoding='latin1', sep=';', on_bad_lines='warn')
    print("Dataset cargado exitosamente desde GitHub.")
    print(f"Dimensiones: {df.shape}")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")

Dataset cargado exitosamente desde GitHub.
Dimensiones: (732, 15)


### <font size= 12 color="lightgreen" >Explorando el dataset <font>

In [ ]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Text', 'Sentiment', 'Timestamp', 'User',
       'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month',
       'Day', 'Hour'],
      dtype='object')

In [ ]:
df.sample(3)

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
587,25,25,Brunch del domingo con amigos.,Positivo,22-01-2023 12:00,BrunchBuddy,Instagram,#Brunch #Amigos,15,30,Reino Unido,2023,1,22,12
700,457,461,"Vagando en el laberinto de la traiciÃ³n, los m...",TraiciÃ³n,05-01-2021 16:20,MazeWanderer,Instagram,#TraiciÃ³n #CerrandoMuros,15,30,Alemania,2021,1,5,16
329,307,311,"Ojos envidiosos fijos en el premio dorado, una...",Envidioso,28-02-2021 22:45,GildedHeartache,Facebook,#Envidioso #PremioDorado,20,40,EE.UU,2021,2,28,22


### <font size=12 color=lightgreen> Crear dataframe filtrado </font>

In [ ]:
#Renombrar columna Text por Texto
df.rename(columns={'Text': 'Texto'}, inplace=True)

# Crar df_filtrado con columnas Texto y Sentiment
df_filtrado = df[['Texto', 'Sentiment']]

# Eliminar registros nulos y datos nulos
df_filtrado.dropna(inplace=True)
df_filtrado = df_filtrado[df_filtrado['Texto'].notna()]

df_filtrado.sample(3)

,Texto,Sentiment
379,SoÃ±ar despierto con el prÃ³ximo baile de grad...,ExcitaciÃ³n
636,Reverencia por el arte exhibido en un museo.,Reverencia
445,OrganizÃ³ una exposiciÃ³n de fotografÃ­as que ...,Gratitud


### <font size=12 color=lightgreen>Limpiar textos</font>

In [ ]:
def pre_proccess_text(texto):
    # 1. Convertir a minúsculas
    texto = texto.lower()

    # 2. Normaliza el texto para separar las tildes de las letras
    texto = unicodedata.normalize('NFD', texto)

    # 3. Filtra y se queda solo con los caracteres que no son tildes
    texto = texto.encode('ascii', 'ignore').decode("utf-8")

    # 4. Eliminar URLs (http, https, www)
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto)

    # 5. Eliminar hashtags (#Haghtag)
    # El patrón r'#\w+' busca el símbolo # seguido de caracteres alfanuméricos
    texto = re.sub(r'#\w+', '', texto)

    # 6. Eliminar menciones (@usuario)
    texto = re.sub(r'@\w+', '', texto)

    # 7. Eliminar caracteres especiales y números (opcional, según tu criterio)
    texto = re.sub(r'[^\w\s]', '', texto)

    # 8 Eliminar emojis
    texto = re.sub(r'[^\x00-\x7F]+', '', texto)


    # 8. Eliminar espacios extra
    texto = texto.strip()

    return texto



# Aplicar al DataFrame
df_filtrado['Texto_Limpio'] = df_filtrado['Texto'].apply(pre_proccess_text)

# Mostrar un ejemplo del antes y después
print(df_filtrado[['Texto', 'Texto_Limpio']].head())


                                               Texto  \
0  Abrumado por el peso del mundo, Atlas con los ...   
1  Abrumado por la cacofonÃ­a de las expectativas...   
2  Abrumado por el laberinto de expectativas, un ...   
3  Abrumado por el apoyo recibido durante un desa...   
4  Llega el aburrimiento, el dÃ­a se siente infin...   

                                        Texto_Limpio  
0  abrumado por el peso del mundo atlas con los h...  
1  abrumado por la cacofonaa de las expectativas ...  
2  abrumado por el laberinto de expectativas un m...  
3  abrumado por el apoyo recibido durante un desa...  
4  llega el aburrimiento el daa se siente infinit...  


### <font size=12 color=lightgreen> Categorizar sentimientos </font>

In [ ]:
# 1. Definimos las listas de sentimientos según su categoría


# Ver todos los sentimientos únicos para saber qué agrupar
print(df_filtrado['Sentiment'].unique())


['Abrumado' 'Aburrimiento' 'AceptaciÃ³n' 'AdmiraciÃ³n' 'AdoraciÃ³n'
 'Adrenalina' 'Afecto' 'Agotamiento' 'Agradecido' 'Agridulce'
 'Aislamiento' 'AlegrÃ\xada' 'AlegrÃ\xada al hornear'
 'AlegrÃ\xada festiva' 'AlegrÃ\xada juguetona' 'Alivio' 'Amabilidad'
 'Amable' 'Amar' 'Amargura' 'Ambivalencia' 'Amistad' 'Amor perdido'
 'Angustia' 'Anhelo' 'Ã\x81nimo' 'Ansiedad' 'AnticipaciÃ³n' 'ApreciaciÃ³n'
 'Aprensivo' 'ArmonÃ\xada' 'Arrepentimiento' 'Asco' 'Asombro' 'Aventura'
 'Aventura Culinaria' 'BendiciÃ³n' 'CÃ¡lculo errÃ³neo' 'Calma' 'Capricho'
 'CautivaciÃ³n' 'Cazador de sueÃ±os' 'CelebraciÃ³n' 'Celos' 'Celoso'
 'Chispa' 'Colorido' 'Comodidad' 'CompasiÃ³n' 'Compasivo' 'Compromiso'
 'ConexiÃ³n' 'Confiado' 'Confianza' 'ConfusiÃ³n' 'Consciencia' 'Consuelo'
 'ContemplaciÃ³n' 'Contentamiento' 'Creatividad'
 'Creatividad de la pasarela' 'Cumplimiento' 'Curiosidad'
 'De espÃ\xadritu libre' 'DecepciÃ³n' 'DesafÃ\xado' 'Desamor'
 'Descubrimiento' 'DesesperaciÃ³n' 'Deslumbrar' 'Despectivo'
 'Determinaci

In [ ]:
# 1. SENTIMIENTOS POSITIVOS (Bienestar, éxito, alegría)
positivos = [
    'Aceptacion', 'Admiracion', 'Adoracion', 'Adrenalina', 'Afecto', 'Agradecido',
    'Alegria', 'Alegria al hornear', 'Alegria festiva', 'Alegria juguetona', 'Alivio',
    'Amabilidad', 'Amable', 'Amar', 'Amistad', 'Animo', 'Apreciacion', 'Armonia',
    'Asombro', 'Aventura', 'Aventura culinaria', 'Bendicion', 'Calma', 'Capricho',
    'Cautivacion', 'Cazador de suenos', 'Celebracion', 'Chispa', 'Colorido', 'Comodidad',
    'Compasion', 'Compasivo', 'Compromiso', 'Conexion', 'Confiado', 'Confianza',
    'Consciencia', 'Consuelo', 'Contentamiento', 'Creatividad', 'Creatividad de la pasarela',
    'Cumplimiento', 'De espiritu libre', 'Descubrimiento', 'Deslumbrar', 'Determinacion',
    'Disfrute', 'Diversion', 'Elacion', 'Elegancia', 'Emocion', 'Emocionado', 'Empatico',
    'Empoderamiento', 'Encantamiento', 'Encanto', 'Energia', 'Entusiasmo', 'Esfuerzo renovado',
    'Esperanza', 'Euforia', 'Excitacion', 'Exito', 'Exploracion', 'Explosion artistica',
    'Extasis', 'Fascinante', 'Felicidad', 'Feliz', 'Grandeza', 'Gratitud', 'Hipnotico',
    'Iconico', 'Imaginacion', 'Inmersion', 'Inspiracion', 'Inspiracion creativa', 'Inspirado',
    'Intriga', 'Jugueton', 'La belleza de la naturaleza', 'La libertad del oceano', 'Libertad',
    'Lleno de alegria', 'Logro', 'Magia de invierno', 'Maravilla', 'Maravilla celestial',
    'Melodico', 'Motivacion', 'Optimismo', 'Orgullo', 'Orgulloso', 'Positividad', 'Positivo',
    'Reconfortante', 'Rejuvenecimiento', 'Resiliencia', 'Resplandor', 'Reunion alegre',
    'Reverencia', 'Romance', 'Satisfaccion', 'Serenidad', 'Tranquilidad', 'Triunfo',
    'Vibrancia', 'Viaje emocionante'
]


# 2. SENTIMIENTOS NEGATIVOS (Dolor, ira, miedo, estrés)
negativos = [
    'Abrumado', 'Aburrimiento', 'Agotamiento', 'Agridulce', 'Aislamiento', 'Amargura',
    'Amor perdido', 'Angustia', 'Anhelo', 'Ansiedad', 'Aprensivo', 'Arrepentimiento',
    'Asco', 'Celos', 'Celoso', 'Decepcion', 'Desafio', 'Desamor', 'Desesperacion',
    'Despectivo', 'Devastado', 'Dolor', 'Enojo', 'Entumecimiento', 'Envidia', 'Envidiar',
    'Envidioso', 'Frustracion', 'Frustrado', 'Impotencia', 'Intimidacion', 'Lastima',
    'Malo', 'Melancolia', 'Miedo', 'Negativo', 'Obstaculo', 'Odiar', 'Oscuridad', 'Pena',
    'Perdida', 'Presion', 'Resentimiento', 'Soledad', 'Sufrimiento', 'Temeroso', 'Temor',
    'Tormenta emocional', 'Traicion', 'Tristeza', 'Tristezaza', 'Verguenza'
]

# 3. SENTIMIENTOS NEUTRALES (O "Grises" que no definen éxito/fracaso)
# Aquí incluimos "Confuso" (Blender) y otros estados contemplativos
neutros = [
    'Ambivalencia', 'Anticipacion', 'Calculo erroneo', 'Confusion', 'Confuso',
    'Contemplacion', 'Curiosidad', 'Indiferencia', 'Neutral', 'Nostalgia',
    'Odisea culinaria', 'Pensive', 'Preguntarse', 'Reflexion', 'Restos',
    'Suspenso', 'Susurros del pasado', 'Travieso', 'Viaje', 'Viaje interior',
    'Visualizando la historia'
]

### <font color=lightgreen size=12>Función para categorizar sentimiento</font>

In [ ]:
# 1. Función de categorización de sentimiento
def categorizar_sentimiento(sentimiento):
    # Limpiamos espacios en blanco y estandarizamos a título
    sent = str(sentimiento).strip().title()

    if sent in positivos:
        return 'Positivo'
    elif sent in negativos:
        return 'Negativo'
    else:
        # Por defecto, lo que no conocemos o es ambiguo va a Neutral para el MVP
        return 'Neutral'


In [ ]:
# 2. Aplicamos la función a tu columna 'Sentimiento'
df_filtrado['Sentimiento_Final'] = df_filtrado['Sentiment'].apply(categorizar_sentimiento)

# 3. Verificamos cómo quedó la distribución
print(df_filtrado['Sentimiento_Final'].value_counts())

Sentimiento_Final
Neutral     330
Positivo    250
Negativo    152
Name: count, dtype: int64


### <font size=12 color=lightgreen> Exportar</font>

In [ ]:
# Eliminar posibles nulos generados tras la limpieza
df_entrega = df_filtrado.dropna(subset=['Texto_Limpio'])
df_entrega = df_entrega[df_entrega['Texto_Limpio'].str.strip() != ""]

In [ ]:
# Seleccionar solo lo necesario
df_entrega = df_entrega[['Texto_Limpio', 'Sentimiento_Final']]
df_entrega.sample(5)

,Texto_Limpio,Sentimiento_Final
666,destrozados por los ecos de un sueao destrozad...,Negativo
353,la emocian aumenta por una fiesta de cumpleaao...,Neutral
558,cocina con axito una comida gourmet para la fa...,Positivo
63,organiza un evento de pintura comunitario conv...,Neutral
577,explorando las joyas ocultas de la ciudad,Positivo


In [ ]:
# Exportar
df_entrega.to_csv(f'dataset_listo_para_ML.csv', index=False)

print("Dataset exportado exitosamente.")

Dataset exportado exitosamente.


### <font size=12 color=lightgreen>Observación</font>

 Se realizó una normalización idiomática manual asistida por Excel para corregir traducciones y unificar el idioma del corpus, mejorando la coherencia semántica previa al entrenamiento del modelo.

### <font size=12 color=lightgreen>Importación del DataFrame limpio</font>

In [ ]:
# URL del archivo RAW en GitHub 
url_dataset_limpio = 'https://github.com/ml-punto-tech/sentiment-api/raw/refs/heads/main/data-science/datasets/dataset_listo_para_ML.csv'

# Mantenemos tus parámetros de encoding y separador
try:
    df_entrega = pd.read_csv(url_dataset_limpio, encoding='latin1', sep=',', on_bad_lines='warn')
    print("Dataset cargado exitosamente desde GitHub.")
    print(f"Dimensiones: {df.shape}")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")
df_entrega.head(5)

Dataset cargado exitosamente desde GitHub.
Dimensiones: (732, 15)


,Texto_Limpio,Sentimiento_Final
0,abrumado por el peso del mundo atlas con los h...,Negativo
1,abrumado por la cacofonia de las expectativas ...,Negativo
2,abrumado por el laberinto de expectativas un m...,Negativo
3,abrumado por el apoyo recibido durante un desa...,Negativo
4,llega el aburrimiento el dia se siente infinit...,Negativo


In [ ]:
df_entrega.rename({'Texto_Limpio':'texto'},axis=1,inplace=True)
df_entrega.rename({'Sentimiento_Final':'sentimiento'},axis=1,inplace=True)

In [ ]:
df_entrega.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   texto        732 non-null    object
 1   sentimiento  732 non-null    object
dtypes: object(2)
memory usage: 11.6+ KB


In [ ]:
df=pd.DataFrame(df_entrega)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   texto        732 non-null    object
 1   sentimiento  732 non-null    object
dtypes: object(2)
memory usage: 11.6+ KB


In [ ]:
# Descargar palabras vacías
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    texto = texto.lower() # Minúsculas
    texto = re.sub(r'[^\w\s]', '', texto) # Eliminar puntuación
    texto = " ".join([word for word in texto.split() if word not in stop_words]) # Quitar stopwords
    return texto

# Aplicar a tu dataset
df['texto'] = df['texto'].apply(limpiar_texto)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df

,texto,sentimiento
0,abrumado peso mundo atlas hombros temblorosos ...,Negativo
1,abrumado cacofonia expectativas alma ahogada t...,Negativo
2,abrumado laberinto expectativas minotauro pres...,Negativo
3,abrumado apoyo recibido desafio personal,Negativo
4,llega aburrimiento dia infinitamente aburrido,Negativo
...,...,...
727,meciendose vibraciones reggae concierto tribut...,Neutral
728,medio paginas cautivadora novela misterio lect...,Neutral
729,explorando universo interior sesion meditacion...,Neutral
730,capturar esencia mercado bullicioso fotografo ...,Positivo


## Balanceo del Dataset, TF-IDF, Modelo, Métricas y Serialización

### Instalación de `imblearn`

Primero, necesitamos instalar la librería `imblearn`, que proporciona herramientas para manejar datasets desbalanceados, incluyendo la técnica SMOTE para sobremuestreo.

In [ ]:
get_ipython().system('pip install imblearn')
print("Librería 'imblearn' instalada exitosamente.")

Librería 'imblearn' instalada exitosamente.


### Separación de Características y Target

Ahora, separaremos las características (el texto limpio) y la variable objetivo (el sentimiento) de nuestro DataFrame `df`. También mostraremos la distribución inicial de las clases para ver el desbalanceo.

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import joblib
import json
import pandas as pd

# Separar características (X) y variable objetivo (y)
X = df['texto']
y = df['sentimiento']

# Verificar la distribución inicial de las clases
print("Distribución inicial de las clases:")
print(y.value_counts())

Distribución inicial de las clases:
sentimiento
Neutral     330
Positivo    250
Negativo    152
Name: count, dtype: int64


### División de Datos (Entrenamiento y Prueba) y Vectorización TF-IDF

Es crucial dividir el dataset en conjuntos de entrenamiento y prueba *antes* de aplicar SMOTE para evitar la fuga de datos (data leakage). Luego, transformaremos los textos en vectores numéricos usando `TfidfVectorizer`.

In [ ]:
# Dividir el dataset en conjuntos de entrenamiento y prueba ANTES de aplicar SMOTE
X_train_unbalanced, X_test, y_train_unbalanced, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTamaño del conjunto de entrenamiento (desbalanceado): {len(X_train_unbalanced)} muestras")
print(f"Tamaño del conjunto de prueba: {len(X_test)} muestras")
print(f"Distribución de clases en el conjunto de entrenamiento (desbalanceado):\n{y_train_unbalanced.value_counts()}")
print(f"Distribución de clases en el conjunto de prueba:\n{y_test.value_counts()}")

# Inicializar TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Limitando las características para eficiencia

# Ajustar y transformar X_train_unbalanced, y transformar X_test
X_train_tfidf_unbalanced = tfidf_vectorizer.fit_transform(X_train_unbalanced)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("\nVectorización TF-IDF completada en la división desbalanceada.")
print(f"Forma de X_train_tfidf_unbalanced: {X_train_tfidf_unbalanced.shape}")
print(f"Forma de X_test_tfidf: {X_test_tfidf.shape}")


Tamaño del conjunto de entrenamiento (desbalanceado): 585 muestras
Tamaño del conjunto de prueba: 147 muestras
Distribución de clases en el conjunto de entrenamiento (desbalanceado):
sentimiento
Neutral     264
Positivo    200
Negativo    121
Name: count, dtype: int64
Distribución de clases en el conjunto de prueba:
sentimiento
Neutral     66
Positivo    50
Negativo    31
Name: count, dtype: int64

Vectorización TF-IDF completada en la división desbalanceada.
Forma de X_train_tfidf_unbalanced: (585, 2251)
Forma de X_test_tfidf: (147, 2251)


### Balanceo del Conjunto de Entrenamiento con SMOTE

Ahora aplicaremos SMOTE solo al conjunto de entrenamiento vectorizado (`X_train_tfidf_unbalanced`) para balancear las clases, generando muestras sintéticas para las clases minoritarias.

In [ ]:
# Inicializar SMOTE para balancear el conjunto de datos de ENTRENAMIENTO
smote = SMOTE(random_state=42)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf_unbalanced, y_train_unbalanced)

print("\nDistribución de clases después de SMOTE en los datos de entrenamiento:")
print(y_train.value_counts())

print(f"Forma de X_train_tfidf después de SMOTE: {X_train_tfidf.shape}")


Distribución de clases después de SMOTE en los datos de entrenamiento:
sentimiento
Neutral     264
Positivo    264
Negativo    264
Name: count, dtype: int64
Forma de X_train_tfidf después de SMOTE: (792, 2251)


### Entrenamiento del Modelo de Regresión Logística

Entrenaremos un modelo de Regresión Logística utilizando los datos de entrenamiento balanceados y vectorizados.

In [ ]:
# Entrenar el Modelo de Regresión Logística
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

print("\nModelo de Regresión Logística entrenado.")

### Evaluación del Modelo

Evaluaremos el rendimiento del modelo en el conjunto de prueba utilizando métricas clave como accuracy, precision, recall y F1-score.

In [ ]:
# Evaluar el Modelo
y_pred = model.predict(X_test_tfidf)
y_pred_proba = model.predict_proba(X_test_tfidf)

print("\nEvaluación del Modelo:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"Precision (ponderada): {precision_score(y_test, y_pred, average='weighted'):.2f}")
print(f"Recall (ponderado): {recall_score(y_test, y_pred, average='weighted'):.2f}")
print(f"F1-Score (ponderado): {f1_score(y_test, y_pred, average='weighted'):.2f}")
print("\nReporte de Clasificación:\n", classification_report(y_test, y_pred))

### Serialización del Modelo y Vectorizador

Guardaremos el modelo entrenado y el objeto `TfidfVectorizer` utilizando `joblib` para poder reutilizarlos más tarde en la API de predicción.

In [ ]:
# Serializar el Modelo y el Vectorizador
joblib.dump(model, '/content/modelo_sentimientos.pkl')
joblib.dump(tfidf_vectorizer, '/content/vectorizador.pkl')

print("\nModelo y vectorizador guardados exitosamente en '/content/modelo_sentimientos.pkl' y '/content/vectorizador.pkl'.")

### Prueba del Modelo con Salida JSON

Crearemos una función para probar el modelo con nuevas reseñas de texto. Esta función preprocesará el texto, lo vectorizará con el `TfidfVectorizer` guardado, realizará una predicción y devolverá el resultado en formato JSON, incluyendo la previsión y la probabilidad de la clase predicha.

In [ ]:
# Recargar el modelo y el vectorizador para probar (como si fuera una nueva sesión/API)
loaded_model = joblib.load('/content/modelo_sentimientos.pkl')
loaded_vectorizer = joblib.load('/content/vectorizador.pkl')

def predict_sentiment_json(text_review):
    # Preprocesamiento (igual que para los datos de entrenamiento)
    # Asumiendo que `pre_proccess_text` y `limpiar_texto` están definidos en celdas anteriores
    cleaned_text = pre_proccess_text(text_review)
    cleaned_text = limpiar_texto(cleaned_text)

    # Vectorizar el texto limpio
    text_vectorized = loaded_vectorizer.transform([cleaned_text])

    # Predecir el sentimiento
    prediction = loaded_model.predict(text_vectorized)[0]

    # Predecir las probabilidades
    probabilities = loaded_model.predict_proba(text_vectorized)[0]
    class_labels = loaded_model.classes_
    # Asegurar el mapeo correcto de probabilidades a etiquetas
    prob_dict = {label: round(prob * 100, 2) for label, prob in zip(class_labels, probabilities)}

    # Obtener la probabilidad de la clase predicha
    predicted_prob = prob_dict[prediction]

    result = {
        "prevision": prediction,
        "probabilidad": predicted_prob
    }
    return json.dumps(result, indent=4)

# Ejemplos de uso de la función de predicción
new_review1 = "Me gusta la actitud del personal"
new_review2 = "Me dio mucho sueno."
new_review3 = "La situación es complicada, no sé qué pensar."

print(f"\nPredicción para '{new_review1}':")
print(predict_sentiment_json(new_review1))

print(f"\nPredicción para '{new_review2}':")
print(predict_sentiment_json(new_review2))

print(f"\nPredicción para '{new_review3}':")
print(predict_sentiment_json(new_review3))

In [ ]:
from sklearn.pipeline import Pipeline

# 1. Creamos un Pipeline manual con el vectorizador y el modelo YA ENTRENADOS
# Esto "engaña" al sistema para empaquetarlos juntos sin re-entrenar
pipeline_exportacion = Pipeline([
    ('tfidf', tfidf_vectorizer),  # Tu vectorizador ya ajustado
    ('clf', model)                # Tu modelo ya entrenado con SMOTE
])

# 2. Guardamos SOLO este pipeline
# Este archivo contiene AMBOS (el diccionario de palabras y la matemática del modelo)
joblib.dump(pipeline_exportacion, 'modelo_entrenado.joblib')

print("✅ Pipeline guardado exitosamente como 'modelo_entrenado.joblib'")
print("   (Este archivo está listo para usarse en tu API sin cambios)")